In [1]:
# This time, let's test auto tuning using hyperopt.

In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM

import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

import gc
from tensorflow.python.keras import backend as K

K.clear_session()
gc.collect()

Using TensorFlow backend.


22

In [3]:
def model(x_train, y_train):
    model = Sequential()
    model.add(Dense(x_train.shape[1],
                   input_shape=(x_train.shape[1],9)))
    model.add(Dropout({{uniform(0, 0.5)}}))
    model.add(LSTM({{choice([9, 16, 32, 64])}}, 
                   return_sequences=False))
    model.add(Dropout({{uniform(0, 0.5)}}))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['acc'])
    
    history = model.fit(x_train,
              y_train,
              epochs = {{choice([16, 32 ,64])}},
              batch_size = {{choice([32 ,64 ,128])}},
              validation_split = 0.1,
              shuffle = False)
    
    val_acc = np.amax(history.history['val_acc'])
    print(f'Best epoch acc:{val_acc}')
    
    return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
# I put dropout layers on after and before LSTM layers this time

In [4]:
def data():
    df = pd.read_csv('../csv/keisuke2.csv',header=0)
    data = df[(df['STATE'] >= 2) & (df['STATE'] <= 5)]
    X = data[['GSR', 'Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta', 'Low_Gamma', 'Mid_Gamma']].values
    mean = X.mean(axis = 0)
    std = X.std(axis = 0)
    
    # Train data
    data = df[(df['STATE'] >= 2) & (df['STATE'] <= 3)]
    X = data[['GSR', 'Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta', 'Low_Gamma', 'Mid_Gamma']].values
    X = (X - mean) / std
    y = data['STATE']
    x_train, y_train = create_data(X, y)
    
    # Test data
    data = df[(df['STATE'] >= 4) & (df['STATE'] <= 5)]
    X = data[['GSR', 'Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta', 'Low_Gamma', 'Mid_Gamma']].values
    X = (X - mean) / std
    y = data['STATE']
    x_test, y_test = create_data(X, y)
    
    return x_train, y_train, x_test, y_test, mean, std

In [5]:
def create_data(X, y):
    time_step=30;
    x_data, y_data = [], []

    for i in range(0, len(X) -time_step, 1):
        #x_data
        tmp = X[i:(i+time_step)].tolist()
        x_data.append(tmp)
        #y_data
        labels = y[i:(i+time_step)]
        mode = collections.Counter(labels).most_common()[0][0]
        if mode % 2 == 0:
            y_data.append(1)
        if mode % 2 == 1:
            y_data.append(0)
        
    return np.array(x_data), np.array(y_data).reshape(-1,1)

In [6]:
def mean_std(X):
    result = np.array(X, dtype=float)
    mean = result.mean(axis = 0)
    result -= mean
    std = result.std(axis = 0)
    result /= std 
    
    return result

In [7]:
import time
start = time.time()
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=3,
                                      eval_space=True,
                                      trials=Trials(),
                                      functions=[create_data, mean_std],
                                      notebook_name='test_auto_tuning')
x_train, y_train, x_test, y_test, mean, std = data()
test_loss, test_acc = best_model.evaluate(x_test, y_test)

>>> Imports:
#coding=utf-8

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Activation, Dropout, LSTM
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import collections
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import gc
except:
    pass

try:
    from tensorflow.python.keras import backend as K
except:
    pass

try:
    import time
except:
    pass

try:
    from nltk import ConfusionMatrix
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 0.5),
        

2021-08-18 18:45:13.761957: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 972000 exceeds 10% of free system memory.


1/7 [===>..........................]                 
 - ETA: 0s - loss: 0.6812 - acc: 0.6406              
                                                     
2/7 [=======>......................]                 
 - ETA: 0s - loss: 0.6628 - acc: 0.6016              
                                                     
3/7 [===========>..................]                 
 - ETA: 0s - loss: 0.6100 - acc: 0.6667              
                                                     
4/7 [================>.............]                 
 - ETA: 0s - loss: 0.5944 - acc: 0.6836              
                                                     
5/7 [====================>.........]                 
 - ETA: 0s - loss: 0.6161 - acc: 0.6625              
                                                     
6/7 [========================>.....]                 
 - ETA: 0s - loss: 0.6048 - acc: 0.6797              
                                                     
7/7 [=======================

 - ETA: 0s - loss: 0.3401 - acc: 0.9375              
                                                     
3/7 [===========>..................]                 
 - ETA: 0s - loss: 0.3103 - acc: 0.9531              
                                                     
4/7 [================>.............]                 
 - ETA: 0s - loss: 0.3379 - acc: 0.9375              
                                                     
5/7 [====================>.........]                 
 - ETA: 0s - loss: 0.3576 - acc: 0.9250              
                                                     
6/7 [========================>.....]                 
 - ETA: 0s - loss: 0.3632 - acc: 0.9245              
                                                     
7/7 [==============================]                 
 - ETA: 0s - loss: 0.3688 - acc: 0.9284              
                                                     
7/7 [==============================]                 
 - 1s 88ms/step - loss: 0.36

                                                     
4/7 [================>.............]                 
 - ETA: 0s - loss: 0.1855 - acc: 0.9805              
                                                     
5/7 [====================>.........]                 
 - ETA: 0s - loss: 0.2054 - acc: 0.9812              
                                                     
6/7 [========================>.....]                 
 - ETA: 0s - loss: 0.2147 - acc: 0.9818              
                                                     
7/7 [==============================]                 
 - ETA: 0s - loss: 0.2199 - acc: 0.9827              
                                                     
7/7 [==============================]                 
 - 1s 88ms/step - loss: 0.2199 - acc: 0.9827 - val_loss: 1.0325 - val_acc: 0.3111

Epoch 14/64                                          
1/7 [===>..........................]                 
 - ETA: 0s - loss: 0.2173 - acc: 0.9844              

...

 - ETA: 0s - loss: 0.3461 - acc: 0.9728                                         
                                                                                
4/4 [==============================]                                            
 - 0s 110ms/step - loss: 0.3461 - acc: 0.9728 - val_loss: 0.3561 - val_acc: 1.0000

Epoch 12/16                                                                     
1/4 [======>.......................]                                            
 - ETA: 0s - loss: 0.3097 - acc: 1.0000                                         
                                                                                
2/4 [==============>...............]                                            
 - ETA: 0s - loss: 0.3414 - acc: 0.9492                                         
                                                                                
3/4 [=====================>........]                                            
 - ETA: 0s - loss: 0.3346

4/4 [==============================]                             
 - ETA: 0s - loss: 0.5168 - acc: 0.8988                          
                                                                 
4/4 [==============================]                             
 - 1s 135ms/step - loss: 0.5168 - acc: 0.8988 - val_loss: 0.7105 - val_acc: 0.5111

Epoch 3/32                                                       
1/4 [======>.......................]                             
 - ETA: 0s - loss: 0.4546 - acc: 0.9609                          
                                                                 
2/4 [==============>...............]                             
 - ETA: 0s - loss: 0.4321 - acc: 0.9531                          
                                                                 
3/4 [=====================>........]                             
 - ETA: 0s - loss: 0.4494 - acc: 0.9531                          
                                                          

 - ETA: 0s - loss: 0.1564 - acc: 0.9901                          
                                                                 
4/4 [==============================]                             
 - 1s 135ms/step - loss: 0.1564 - acc: 0.9901 - val_loss: 0.2356 - val_acc: 1.0000

Epoch 11/32                                                      
1/4 [======>.......................]                             
 - ETA: 0s - loss: 0.1045 - acc: 1.0000                          
                                                                 
2/4 [==============>...............]                             
 - ETA: 0s - loss: 0.1287 - acc: 0.9844                          
                                                                 
3/4 [=====================>........]                             
 - ETA: 0s - loss: 0.1312 - acc: 0.9896                          
                                                                 
4/4 [==============================]                      

                                                                 
4/4 [==============================]                             
 - 1s 126ms/step - loss: 0.0584 - acc: 0.9975 - val_loss: 0.0377 - val_acc: 1.0000

Epoch 19/32                                                      
1/4 [======>.......................]                             
 - ETA: 0s - loss: 0.0459 - acc: 1.0000                          
                                                                 
2/4 [==============>...............]                             
 - ETA: 0s - loss: 0.0630 - acc: 0.9961                          
                                                                 
3/4 [=====================>........]                             
 - ETA: 0s - loss: 0.0550 - acc: 0.9974                          
                                                                 
4/4 [==============================]                             
 - ETA: 0s - loss: 0.0539 - acc: 0.9975                   

4/4 [==============================]                             
 - 1s 143ms/step - loss: 0.0341 - acc: 0.9975 - val_loss: 0.0167 - val_acc: 1.0000

Epoch 27/32                                                      
1/4 [======>.......................]                             
 - ETA: 0s - loss: 0.0298 - acc: 1.0000                          
                                                                 
2/4 [==============>...............]                             
 - ETA: 0s - loss: 0.0436 - acc: 0.9922                          
                                                                 
3/4 [=====================>........]                             
 - ETA: 0s - loss: 0.0365 - acc: 0.9948                          
                                                                 
4/4 [==============================]                             
 - ETA: 0s - loss: 0.0357 - acc: 0.9951                          
                                                          

In [8]:
print(f'Elapsed time:{time.time() - start}')
print(best_run)
print(test_loss, test_acc)

Elapsed time:120.43572854995728
{'Dropout': 0.25764043248072455, 'Dropout_1': 0.21261430843422813, 'LSTM': 16, 'batch_size': 128, 'epochs': 16}
0.29087716341018677 0.9155555367469788


In [9]:
# It took almost 2 mins to test all of setting patterns
# And test loss and acc are even better than manual tuning :D

In [10]:
result = best_model.predict(x_test)
for i in range(len(result)):
    # Focus state if result is over 0.5
    if result[i] >= 0.5:
        result[i] = 1
    else:
        result[i] = 0

In [11]:
test = y_test.reshape(-1,).tolist()
result = result.reshape(-1,).tolist()
from nltk import ConfusionMatrix
print(ConfusionMatrix(test, result))

  |   0   1 |
--+---------+
0 |<224>  . |
1 |  38<188>|
--+---------+
(row = reference; col = test)



In [12]:
# Confusion matrix also shows better result
# Now let's proceed to real time detection